<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/13_rnn/04_Seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder-Decoderによる計算機作成

---
## エンコーダ・デコーダ

リカレントニューラルネットワークは，系列データ内の関連性を内部状態として保持することができます．
この内部状態を利用して，新たな出力ができるようにした構造としてエンコーダ・デコーダがあります．
エンコーダ側に系列データを入力して，中間層では系列データ内の関連性を内部状態を形成します．
デコーダ側には内部状態を与えることで，内部状態を反映した何かしらの結果を出力します．
この応用が，google 翻訳などの機械翻訳です．

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/04_Seq2seq/Seq2Seq.png?raw=true" width = 100%>



##計算機の実装
ここでは，エンコーダ・デコーダ構造で計算機（足し算）を作ってみます．
このエンコーダ・デコーダ構造のことをSeq2seqと呼びます．

###データローダの作成
まず，データローダを用意します．データは0から9までの数字と加算記号，開始，終了のフラグです．また，３桁の数字の足し算を行うため，各桁の値を１つずつランダムに生成して連結しています．


In [ ]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

word2id = {str(i): i for i in range(10)}
word2id.update({"<pad>": 10, "+": 11, "<eos>": 12})
id2word = {v: k for k, v in word2id.items()}

class CalcDataset(torch.utils.data.Dataset):

    def transform(self, string, seq_len=7):
        tmp = []
        for i, c in enumerate(string):
            try:
                tmp.append(word2id[c])
            except:
                tmp += [word2id["<pad>"]] * (seq_len - i)
                break
        return tmp

    def __init__(self, data_num, train=True):
        super().__init__()
        self.data_num = data_num
        self.train = train
        self.data = []
        self.label = []

        for _ in range(data_num):
            x = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            y = int("".join([random.choice(list("0123456789")) for _ in range(random.randint(1, 3))] ))
            left = ("{:*<7s}".format(str(x) + "+" + str(y))).replace("*", "<pad>")
            self.data.append(self.transform(left))

            z = x + y
            right = ("{:*<6s}".format(str(z))).replace("*", "<pad>")
            right = self.transform(right, seq_len=5)
            right = [12] + right
            right[right.index(10)] = 12
            self.label.append(right)
        


        self.data = np.asarray(self.data)
        self.label = np.asarray(self.label)

    def __getitem__(self, item):
        d = self.data[item]
        l = self.label[item]
        return d, l

    def __len__(self):
        return self.data.shape[0]

###エンコーダ・デコーダの作成
エンコーダとデコーダを用意します．エンコーダは，ワードエンベディングという特徴表現に変換する層とGRU層から構成されています．デコーダも同様の構造です．エンコーダ側の中間層の値がstateとして出力され，デコーダ側の中間層に入力されます．
エンコーダとデコーダは別々のネットワークとして用意し，それぞれの最適化にはAdamを利用します．

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


embedding_dim = 16
hidden_dim = 128
vocab_size = len(word2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_size=100):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2id["<pad>"])
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)

    def forward(self, indices):
        embedding = self.word_embeddings(indices)
        if embedding.dim() == 2:
            embedding = torch.unsqueeze(embedding, 1)
        _, state = self.gru(embedding, torch.zeros(1, self.batch_size, self.hidden_dim, device=device))
        
        return state


class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_size=100):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=word2id["<pad>"])
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, index, state):
        embedding = self.word_embeddings(index)
        if embedding.dim() == 2:
            embedding = torch.unsqueeze(embedding, 1)
        gruout, state = self.gru(embedding, state)
        output = self.output(gruout)
        return output, state


encoder = Encoder(vocab_size, embedding_dim, hidden_dim, batch_size=100).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, batch_size=100).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2id["<pad>"])

# Initialize opotimizers
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)

###学習
学習を行います．学習データを2万サンプル生成して，データローダに与えます．
学習は200エポック行います．エンコーダの入力は数字または開始・終了・加算記号です．
デコーダの入力は計算結果です．
具体的には，54+37 を行う時，
エンコーダには，まず開始記号を最初に入力し，次に，5, 4, +, 3, 7 を入力します．そして，最後に終了記号を入力します．その時の中間層の情報をhidden_stateとしてエンコーダから受け取ります．
デコーダは，開始記号と中間情報(hidden_state)を最初に入力します，そして，計算結果の9, 1 を入力し，最後に終了記号を入力します．
この時，デコーダは各数字（または記号）の確率をdecoder_outputとして出力します．
decoder_outputは，[バッチサイズ, 1, 各クラス確率]の３次元なので，squeezeによって，[バッチサイズ,  各クラス確率] に次元削減します．
そして，クロスエントロピー誤差関数によって，ロスを求めます．
これを正解の長さ(=5)分繰り返し行い，ロスを累積します．
その後，誤差逆伝播，デコーダ，エンコーダの更新を行います．


In [ ]:
import numpy as np
from time import time

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

batch_size=100
epoch_num = 200

train_data = CalcDataset(data_num = 20000)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

start = time()
for epoch in range(1, epoch_num+1):
    for data, label in train_loader:
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        if use_cuda:
            data = data.cuda()
            label = label.cuda()

        encoder_hidden = encoder(data)
        source = label[:, :-1]
        target = label[:, 1:]
        decoder_hidden = encoder_hidden

        loss = 0
        for i in range(source.size(1)):
            decoder_output, decoder_hidden = decoder(source[:, i], decoder_hidden)
            decoder_output = torch.squeeze(decoder_output)
            loss += criterion(decoder_output, target[:, i])

        # Perform backpropagation
        loss.backward()
        
        # Adjust model weights
        encoder_optimizer.step()
        decoder_optimizer.step()

  
    elapsed_time = time() - start
    if epoch % 10 == 0:
        print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch, loss.item(), elapsed_time))
        
model_name = "seq2seq_calculator_v{}.pt".format(epoch)
torch.save({
    'encoder_model': encoder.state_dict(),
    'decoder_model': decoder.state_dict(),
}, model_name)


Use CUDA: True
torch.Size([100, 13])
torch.Size([100])


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


###評価
次に，学習したモデルを評価をします．テストデータを2000サンプル生成して，データローダに与えます．
ここで，学習時はエンコーダとデコーダのバッチサイズを100としていました．
テスト時は１つずつ行いたいので，エンコーダとデコーダを新たに生成し，学習したパラメータをロードします．
エンコーダ側に計算したい数字（または記号）を入力して中間情報stateを得ます．
デコーダ側に，中間情報stateと開始記号<eos>を入力します．
デコーダ側の出力は数字または記号(token)と中間情報です．
これらを繰り返しデコーダに入力します．<eos>が出力されたら繰り返しは終了です．
出力されたtokenを追加したリストrightを計算結果とします．
計算する式(left)を作成した後，evalでその計算結果が正しいかどうかを判定します．



In [ ]:

batch_size = 1
test_data = CalcDataset(data_num = 2000)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

encoder = Encoder(vocab_size, embedding_dim, hidden_dim, batch_size=1).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, batch_size=1).to(device)

model_name = "seq2seq_calculator_v{}.pt".format(epoch)
checkpoint = torch.load(model_name)
encoder.load_state_dict(checkpoint["encoder_model"])
decoder.load_state_dict(checkpoint["decoder_model"])

accuracy = 0
        
# 評価の実行   
with torch.no_grad():
    for data, label in test_loader:
        if use_cuda:
            data = data.cuda()

        state = encoder(data)

        right = []
        token = "<eos>"
        for _ in range(7):
            index = word2id[token]
            input_tensor = torch.tensor([index], device=device)
            output, state = decoder(input_tensor, state)
            prob = F.softmax(torch.squeeze(output))
            index = torch.argmax(prob.cpu().detach()).item()
            token = id2word[index]
            if token == "<eos>":
                break
            right.append(token)
        right = "".join(right)
        
        x = list(data[0].to('cpu').detach().numpy() )
        try:
            padded_idx_x = x.index(word2id["<pad>"])
        except ValueError:
            padded_idx_x = len(x)
        left = "".join(map(lambda c: str(id2word[c]), x[:padded_idx_x]))



        flag = ["F", "T"][eval(left) == int(right)]
        print("{:>7s} = {:>4s} :{}".format(left, right, flag))
        if flag == "T":
            accuracy += 1
print("Accuracy: {:.2f}".format(accuracy / len(test_loader)))



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


    5+5 =   10 :T
  1+468 =  469 :T
 42+777 =  819 :T
   8+95 =  103 :T
   8+81 =   89 :T
  604+1 =  605 :T
   30+9 =   39 :T
 95+918 = 1013 :T
    0+8 =    8 :T
   33+1 =   34 :T
  3+432 =  435 :T
   97+1 =   98 :T
  577+1 =  578 :T
    6+8 =   14 :T
  732+3 =  735 :T
 301+51 =  352 :T
  192+9 =  201 :T
    2+3 =    5 :T
 625+40 =  665 :T
 65+459 =  524 :T
   60+0 =   60 :T
  22+62 =   84 :T
  193+8 =  201 :T
   3+15 =   18 :T
  103+6 =  119 :F
   9+56 =   65 :T
   0+53 =   53 :T
   4+73 =   77 :T
 89+517 =  606 :T
   9+24 =   33 :T
    9+8 =   17 :T
  589+7 =  596 :T
   9+65 =   74 :T
    5+7 =   12 :T
745+246 =  971 :F
    3+4 =    7 :T
   9+12 =   21 :T
 491+29 =  510 :F
 563+82 =  635 :F
   7+98 =  105 :T
 46+806 =  852 :T
   1+87 =   88 :T
    2+9 =   11 :T
 68+228 =  296 :T
 899+87 =  986 :T
  0+690 =  600 :F
  9+610 =  619 :T
   5+36 =   41 :T
 164+35 =  199 :T
  902+2 =  904 :T
   88+3 =   91 :T
  0+757 =  757 :T
  537+7 =  544 :T
    5+8 =   13 :T
   6+89 =   95 :T
 97+574 = 

#課題
* 足し算だけでなく，色々な四則演算を試そう
* 他のリカレントニューラルネットワークを使って精度比較をしてみよう